In [ ]:
"""TrabalhoIA.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1A33LupY9hRDOSC991fbZgnh-09e3NonP
"""

In [ ]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
!wget -q $url       # baixa o ZIP
!unzip -q bank.zip  # extrai bank-full.csv e outros

In [ ]:
df = pd.read_csv("bank-full.csv", sep=";")
df.head()

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
(ds_train, ds_val, ds_test), info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    as_supervised=True,
    with_info=True,
)
print("Classes:", info.features["label"].names)

In [ ]:
import zipfile
import os
import pandas as pd

Etapa 1: Upload manual do arquivo ZIP no Colab
from google.colab import files
uploaded = files.upload()

In [ ]:
# Etapa 2: Extrair o ZIP externo
outer_zip_path = '/content/bank+marketing.zip'
outer_extract_path = '/content/bank_outer'
with zipfile.ZipFile(outer_zip_path, 'r') as outer_zip:
    outer_zip.extractall(outer_extract_path)

In [ ]:
# Etapa 3: Extrair o ZIP interno (bank.zip)
inner_zip_path = os.path.join(outer_extract_path, 'bank.zip')
inner_extract_path = '/content/bank_data'
with zipfile.ZipFile(inner_zip_path, 'r') as inner_zip:
    inner_zip.extractall(inner_extract_path)

In [ ]:
# Etapa 4: Listar e carregar o CSV principal
print("Arquivos disponíveis:", os.listdir(inner_extract_path))

In [ ]:
df = pd.read_csv(os.path.join(inner_extract_path, 'bank.csv'), sep=';')
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# 1. Converter variáveis binárias para 0/1
df_bin = df.copy()
binary_map = {'yes': 1, 'no': 0}
for col in ['default', 'housing', 'loan', 'y']:
    df_bin[col] = df_bin[col].map(binary_map)

In [ ]:
# 2. One-hot encoding nas variáveis categóricas
categorical_cols = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
df_encoded = pd.get_dummies(df_bin, columns=categorical_cols, drop_first=True)

In [ ]:
# 3. Separar X (features) e y (rótulo)
X = df_encoded.drop('y', axis=1)
y = df_encoded['y']

In [ ]:
# 4. Padronizar colunas numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# 5. Dividir em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Lista de configurações de hiperparâmetros
params_list = [
    {"criterion": "gini", "max_depth": None},
    {"criterion": "entropy", "max_depth": 5},
    {"criterion": "gini", "max_depth": 10, "min_samples_split": 10}
]

In [ ]:
# Lista para guardar resultados
results_dt = []

In [ ]:
for i, params in enumerate(params_list, 1):
    clf = DecisionTreeClassifier(random_state=42, **params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    acc = accuracy_score(y_test, y_pred)

    results_dt.append({
        "Configuração": f"Configuração {i}",
        "Hiperparâmetros": params,
        "Acurácia": acc,
        "Precisão (classe 1)": report['1']['precision'],
        "Recall (classe 1)": report['1']['recall'],
        "F1-score (classe 1)": report['1']['f1-score']
    })

In [ ]:
# Mostrar resultados
import pandas as pd
pd.DataFrame(results_dt)

In [ ]:
"""# Task
Carrega o conjunto de resenhas de livros em português para classificar sentimentos.

Here is all the data you need:
"archive/books_reviews.csv"

## Data loading

### Subtask:
Load the dataset "archive/books_reviews.csv" into a pandas DataFrame.

**Reasoning**:
Carregar a base de resenhas já combinada em um único CSV local e exibir informações básicas.
"""

In [ ]:
import pandas as pd

In [ ]:
try:
    df = pd.read_csv('archive/books_reviews.csv')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('archive/books_reviews.csv', encoding='latin-1')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv('archive/books_reviews.csv', encoding='utf-8')
        except Exception as e:
            print(f"Error loading the file: {e}")
            df = None

In [ ]:
if df is not None:
    display(df.head())
    print(df.shape)
    df.info()

In [ ]:
"""## Data exploration

### Subtask:
Explorar rapidamente a base de resenhas, verificando quantidade de exemplos por classe e tamanho médio dos textos.

**Reasoning**:
Como o conjunto possui apenas duas colunas (texto e rótulo), a análise pode ser simplificada para foco na distribuição das classes e nas estatísticas básicas das resenhas.
"""

In [ ]:
print("Shape of the DataFrame:", df.shape)
print(df['label'].value_counts())

In [ ]:
# calcular o comprimento médio das resenhas
df['text_length'] = df['review_text'].astype(str).apply(len)
print("Tamanho médio do texto:", df['text_length'].mean())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='label', data=df)
plt.title('Distribuição das classes')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df['text_length'], kde=True)
plt.title('Distribuição do tamanho das resenhas')
plt.show()